In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
def extract_features(passwords):
    features = []
    for pwd in passwords:
        features.append([
            len(pwd),                              # Total length
            sum(1 for c in pwd if c.isupper()),    # Uppercase count
            sum(1 for c in pwd if c.islower()),    # Lowercase count
            sum(1 for c in pwd if c.isdigit()),    # Digit count
            sum(1 for c in pwd if not c.isalnum()) # Special chars
        ])
    return np.array(features)

# 2. Load your data (replace with your actual data loading)
data = pd.read_csv("data.csv", on_bad_lines='skip').dropna()

# Map strength to numerical values for XGBoost
label_map = {"Weak": 0, "Medium": 1, "Strong": 2}
data["strength"] = data["strength"].map({0: "Weak", 1: "Medium", 2: "Strong"})  # First map to strings if needed
data["strength"] = data["strength"].map(label_map)  # Then map to numbers

# 3. Create features
X = extract_features(data["password"])
y = data["strength"]

# 4. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Train model
model = XGBClassifier(n_estimators=100, n_jobs=-1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# 6. Prediction function with label mapping back to strings
def predict_strength(password):
    features = extract_features([password])
    pred = model.predict(features)[0]
    return {0: "Weak", 1: "Medium", 2: "Strong"}[pred]

# --- TEST YOUR PASSWORD ---
user_password = input("Enter Password:")
print(f"Password: {user_password} | Strength: {predict_strength(user_password)}")

Model Accuracy: 1.00


Enter Password: Hello1234567


Password: Hello1234567 | Strength: Medium
